# 🧠 Урок 24: Feature Engineering и отбор признаков
**Цель урока:** Научиться создавать новые признаки (feature engineering) и выбирать наиболее важные (feature selection) для улучшения качества моделей машинного обучения. Подходит для новичков.

## 📌 Что такое feature engineering?
- **Feature engineering** — это процесс создания новых признаков из исходных данных, чтобы модель могла лучше находить закономерности [[3]](https://example.com ).
- **Зачем нужен?** Хорошие признаки упрощают обучение, повышают точность и уменьшают переобучение.
- **Аналогия:** Представьте, что вы повар. Feature engineering — как добавление новых ингредиентов в блюдо, чтобы вкус стал лучше.

## 📉 Что такое feature selection (отбор признаков)?
- **Отбор признаков** — это процесс удаления нерелевантных или избыточных признаков для улучшения модели [[4]](https://example.com ).
- **Зачем?** Уменьшает сложность модели, ускоряет обучение, улучшает интерпретируемость.
- **Аналогия:** Feature selection — как убирать лишние ингредиенты из рецепта, чтобы блюдо стало проще и вкуснее.

## 🛠️ Типы feature engineering
### 1. Создание новых признаков
- **Полиномиальные признаки:** Создание комбинаций существующих признаков (например, `x1*x2`).
- **Взаимодействия между признаками:** Суммы, разности, произведения.
- **Категориальные признаки:** One-Hot кодирование, Label Encoding.
- **Пример:** В задаче прогнозирования цены дома добавление признака "площадь_на_человека = площадь / количество_людей" может улучшить модель.

### 2. Преобразование признаков
- **Нормализация:** Приведение к диапазону [0, 1].
- **Стандартизация:** Среднее 0, дисперсия 1.
- **Кодирование категорий:** One-Hot, Ordinal Encoding.
- **Пример:** Преобразование текстовых меток в числа:
  ```python
  df['Gender'] = df['Gender'].map({'Male': 0, 'Female': 1})
  ```

## 🧪 Методы feature selection
### 1. Фильтрация (Filter Methods)
- **Идея:** Выбор признаков на основе статистических тестов (например, корреляция).
- **Методы:**
  - `SelectKBest` — выбирает `k` лучших признаков.
  - `SelectPercentile` — выбирает лучшие проценты признаков.
- **Пример:**
  ```python
  from sklearn.feature_selection import SelectKBest, f_classif
  selector = SelectKBest(score_func=f_classif, k=5)
  X_new = selector.fit_transform(X_train, y_train)
  ```

### 2. Обертки (Wrapper Methods)
- **Идея:** Выбор признаков через обучение модели.
- **Методы:**
  - **RFE (Recursive Feature Elimination):** Последовательно удаляет наименее важные признаки.
  - **GridSearchCV:** Перебор подмножеств признаков.
- **Пример:**
  ```python
  from sklearn.feature_selection import RFE
  from sklearn.ensemble import RandomForestClassifier
  
  model = RandomForestClassifier()
  selector = RFE(model, n_features_to_select=5)
  X_rfe = selector.fit_transform(X_train, y_train)
  ```

### 3. Встроенные методы (Embedded Methods)
- **Идея:** Отбор признаков в процессе обучения модели.
- **Методы:**
  - **L1-регуляризация (Lasso):** Сжимает коэффициенты нерелевантных признаков к нулю.
  - **Random Forest / XGBoost:** Использование `feature_importances_` для оценки важности.
- **Пример:**
  ```python
  from sklearn.ensemble import RandomForestClassifier
  model = RandomForestClassifier(n_estimators=100)
  model.fit(X_train, y_train)
  importance = model.feature_importances_
  ```

## 📊 Практика: Feature Engineering на примере датасета
### Шаг 1: Загрузка данных

In [ ]:
from sklearn.datasets import load_breast_cancer
import pandas as pd

# Загрузка данных
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target
print("Оригинальные признаки:", X.columns.tolist())

### Шаг 2: Создание новых признаков

In [ ]:
import numpy as np

# Создаем новые признаки
X['mean_se'] = X['mean radius'] * X['mean texture']  # Комбинация признаков
X['area_se'] = np.sqrt(X['mean area'])  # Нелинейное преобразование
print("Новые признаки:", X.columns.tolist())

### Шаг 3: Нормализация признаков

In [ ]:
from sklearn.preprocessing import StandardScaler

# Стандартизация
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### Шаг 4: Отбор признаков через SelectKBest

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

# Выбор 10 лучших признаков
selector = SelectKBest(score_func=f_classif, k=10)
X_new = selector.fit_transform(X_scaled, y)
print("Количество признаков после отбора:", X_new.shape[1])

### Шаг 5: Визуализация важности признаков

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Важность через Random Forest
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_scaled, y)
importances = model.feature_importances_

# График важности
indices = np.argsort(importances)[::-1]
plt.figure(figsize=(10, 6))
plt.bar(range(X.shape[1]), importances[indices])
plt.xticks(range(X.shape[1]), X.columns[indices], rotation=90)
plt.title('Важность признаков')
plt.show()

## 📈 Практика: Обучение с отобранными признаками
### Шаг 1: Разделение данных

In [ ]:
from sklearn.model_selection import train_test_split

# Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)

### Шаг 2: Обучение модели

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Обучение
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Оценка
accuracy = accuracy_score(y_test, model.predict(X_test))
print(f'Accuracy: {accuracy:.2f}')

## 📉 Что такое переобучение и как его избежать?
- **Переобучение (Overfitting):** Модель идеально запоминает тренировочные данные, но плохо работает на тестовых.
- **Причины:**
  - Слишком много признаков.
  - Недостаток данных.
- **Как бороться?**
  - Упрощение модели.
  - Добавление регуляризации.
  - Использование feature selection.
- **Пример:**
  ```python
  from sklearn.linear_model import Lasso
  lasso = Lasso(alpha=0.1)
  lasso.fit(X_train, y_train)
  ```

## 📊 Как выбрать лучшие признаки?
- **SelectKBest:** На основе статистики (например, `f_classif`, `mutual_info`).
- **PCA (снижение размерности):** Преобразует признаки в более компактное представление.
- **RFE:** Итеративное удаление наименее важных признаков.
- **Пример:**
  ```python
  from sklearn.decomposition import PCA
  pca = PCA(n_components=5)
  X_pca = pca.fit_transform(X_scaled)
  ```

## 📝 Домашнее задание
**Задача 1:** Добавьте 2 новых признака в датасет (например, произведение двух признаков или квадрат одного из них).
**Задача 2:** Примените метод `SelectKBest` с `k=5` и обучите модель. Сравните accuracy до и после.
**Задача 3:** Напишите отчет (200–300 слов), где:
- Опишите, какие признаки вы добавили.
- Объясните, почему эти признаки могут быть полезны.
- Сравните точность модели до и после feature selection.
- Приведите примеры, как отбор признаков помогает в реальных задачах.

In [ ]:
# Ваш код здесь
from sklearn.feature_selection import SelectKBest, f_classif

# Добавление новых признаков
X['new_feature_1'] = X['mean radius'] * X['mean texture']  # Произведение признаков
X['new_feature_2'] = X['mean area'] ** 2  # Квадрат признака

# Отбор признаков
selector = SelectKBest(score_func=f_classif, k=5)
X_new = selector.fit_transform(X, y)
print("Количество признаков после отбора:", X_new.shape[1])

In [ ]:
# Обучение модели
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)

# Обучение
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Оценка
accuracy = accuracy_score(y_test, model.predict(X_test))
print(f'Accuracy после отбора: {accuracy:.2f}')

## ✅ Рекомендации по выполнению
- **Задача 1:** Используйте комбинации признаков, которые могут отражать скрытые закономерности (например, `площадь_на_человека = площадь / количество_людей`).
- **Задача 2:** Следите за переобучением — если accuracy на train значительно выше, чем на test, уменьшите количество признаков.
- **Подсказка:** Используйте `selector.get_support()` для анализа, какие признаки были отобраны.